In [1]:
# Controle data .gouv
# 1. caracteristics
import pandas as pd
import numpy as np

# Chemins d'accès aux données
repData = '../../dataBrutes.gouv/'
repDataConcat = '../../dataConcat.gouv/'

# Liste des fichiers usagers disponibles
!ls {repData}ca* -lrt
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

ls: -lrt: No such file or directory
../../dataBrutes.gouv/caracteristiques-2017.csv
../../dataBrutes.gouv/caracteristiques-2018.csv
../../dataBrutes.gouv/caracteristiques-2019.csv
../../dataBrutes.gouv/caracteristiques-2020.csv
../../dataBrutes.gouv/caracteristiques_2005.csv
../../dataBrutes.gouv/caracteristiques_2006.csv
../../dataBrutes.gouv/caracteristiques_2007.csv
../../dataBrutes.gouv/caracteristiques_2008.csv
../../dataBrutes.gouv/caracteristiques_2009.csv
../../dataBrutes.gouv/caracteristiques_2010.csv
../../dataBrutes.gouv/caracteristiques_2011.csv
../../dataBrutes.gouv/caracteristiques_2012.csv
../../dataBrutes.gouv/caracteristiques_2013.csv
../../dataBrutes.gouv/caracteristiques_2014.csv
../../dataBrutes.gouv/caracteristiques_2015.csv
../../dataBrutes.gouv/caracteristiques_2016.csv
../../dataBrutes.gouv/carcteristiques-2021.csv
../../dataBrutes.gouv/carcteristiques-2022.csv


In [17]:
df_2005.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87026 entries, 0 to 87025
Data columns (total 16 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Num_Acc  87026 non-null  int64  
 1   an       87026 non-null  int64  
 2   mois     87026 non-null  int64  
 3   jour     87026 non-null  int64  
 4   hrmn     87026 non-null  int64  
 5   lum      87026 non-null  int64  
 6   agg      87026 non-null  int64  
 7   int      87026 non-null  int64  
 8   atm      87026 non-null  int64  
 9   col      87026 non-null  int64  
 10  com      87024 non-null  float64
 11  adr      70238 non-null  object 
 12  gps      27037 non-null  object 
 13  lat      26943 non-null  float64
 14  long     26943 non-null  float64
 15  dep      87026 non-null  int64  
dtypes: float64(3), int64(11), object(2)
memory usage: 10.6+ MB


#### Consolidation de 'caracteristique.csv' - modus operandi
0. Chgt de tous les fichiers 
      - nom des fichiers de 2005 à 2016 : '_'
      - nom des fichiers de 2017 à 2022 : remplacement de '_' par '-'
      - nom des fichiers 2021 et 2022 (manque un 'a') : carcteristiques-2021.csv et carcteristiques-2022.csv
      - séparateur ',' sauf 2009='\t' et 2019 à 2022=';'

1. Variable 'an' à passer de 2 à 4 caractère (ajout 20 en préfixe) de 2005 à 2016
2. Variable 'hrmn', retirer le caractère ':' et forcer le type en int sur fichiers de 2019 à 2022
3. Variable 'gps' manquante de 2019 à 2022 :
      - ajouter la variable avant de concaténer au df principal : lancer les valeurs en Nan pour être iso aux autres fichiers

4. Variable 'Num_Acc' à renommer en 'id_accident' dans tous les fichiers sauf celui de 2022.
5. Lat et long à typer de 'object' --> 'float' (séparateur de décimal =',')
      - 2009 : aucune coordonnée valide : on les force à NaN
      - de 2005 à 2018 (hors 2009): diviser la valeur par 100000
      - de 2019 à 2022 : maintenir en l'état
    
6. Variable 'com' (commune) :
      - 2022 : un accident associé à la com='C' --> corriger la donnée avec '500' qui est le code de la commune
      - de 2005 à 2018 : maintien de la donnée en l'état (float plutôt que int pour gérer les NaN  éventuels)
      - de 2019 à 2022 : garder les 3 caractères de droite et passer en float64
    
7. Variable 'dep' (département) :
      - de 2005 à 2018 : diviser par 10 si %10 sinon laisser en l'état (Corse=201/202, DOM=XXX)
      - de 2019 à 2022 : maintien en l'état en remplaçant les A en '01' et B en '02' (Corse=201/202 au lieu de 2A, 2B)
      - 'dep' est renommée 'dept' dans tous les df
      - Forcer le type de 'str' --> int64
     
8. Ordonner les colonnes de chaque df dans le même ordre
9. Concaténation en un seul fichier
10. Controles
11. Suppression des colonnes inutiles
12. Sauvegarde du df consolidé sous un fichier caracteristiques.csv
    

In [2]:
# 0. chargement et merge de tous les fichiers de 2005 à 2016 --> carac
df_2005 = pd.read_csv(repData + "caracteristiques_2005.csv", encoding='latin-1', sep = ',')
df_2006 = pd.read_csv(repData + "caracteristiques_2006.csv", encoding='latin-1', sep = ',')
df_2007 = pd.read_csv(repData + "caracteristiques_2007.csv", encoding='latin-1', sep = ',')
df_2008 = pd.read_csv(repData + "caracteristiques_2008.csv", encoding='latin-1', sep = ',')
df_2009 = pd.read_csv(repData + "caracteristiques_2009.csv", sep = '\t')
df_2010 = pd.read_csv(repData + "caracteristiques_2010.csv", encoding='latin-1', sep = ',')
df_2011 = pd.read_csv(repData + "caracteristiques_2011.csv", encoding='latin-1', sep = ',')
df_2012 = pd.read_csv(repData + "caracteristiques_2012.csv", encoding='latin-1', sep = ',')
df_2013 = pd.read_csv(repData + "caracteristiques_2013.csv", encoding='latin-1', sep = ',')
df_2014 = pd.read_csv(repData + "caracteristiques_2014.csv", encoding='latin-1', sep = ',')
df_2015 = pd.read_csv(repData + "caracteristiques_2015.csv", encoding='latin-1', sep = ',')
df_2016 = pd.read_csv(repData + "caracteristiques_2016.csv", encoding='latin-1', sep = ',')
df_2017 = pd.read_csv(repData + "caracteristiques-2017.csv", encoding='latin-1', sep = ',')
df_2018 = pd.read_csv(repData + "caracteristiques-2018.csv", encoding='latin-1', sep = ',')
df_2019 = pd.read_csv(repData + "caracteristiques-2019.csv", encoding='latin-1', sep = ';')
df_2020 = pd.read_csv(repData + "caracteristiques-2020.csv", encoding='latin-1', sep = ';')
df_2021 = pd.read_csv(repData + "carcteristiques-2021.csv", encoding='latin-1', sep = ';')
df_2022 = pd.read_csv(repData + "carcteristiques-2022.csv", encoding='latin-1', sep = ';')



In [12]:
df_2019.lat.value_counts()

lat
48,8100000      25
48,7900000      21
43,1213200      14
43,1334900      14
48,9100000      14
48,8000000      12
43,1434000      11
48,9244000      11
48,8200000      11
48,8280945       9
48,7700000       9
43,2800000       8
48,9200000       8
48,9000000       8
 -17,7233790     8
48,7500000       8
43,1250600       7
48,8700000       7
48,8150000       7
 -16,4954170     7
48,8800000       7
48,7940000       6
43,1293100       6
48,7800000       6
48,7980000       6
43,3000000       6
43,2700000       6
43,1265500       6
48,9372920       6
49,0029480       6
48,8176000       6
 -17,5845220     6
48,9333000       5
48,8600000       5
48,7979000       5
48,9063200       5
43,2600000       5
43,2900000       5
 -17,6899310     5
48,9189900       5
48,8840000       5
48,8818900       5
45,6367570       5
48,8620140       5
48,8300000       5
48,9010410       5
48,7400000       5
48,8871300       4
 -21,3434000     4
48,9630000       4
48,8936100       4
43,1806200       4
48,80155

In [3]:
# 1. Variable 'an' à passer de 2 à 4 digit (ajouter 20)  fichiers de 2005 à 2018
df_2005["an"] = df_2005["an"].apply(lambda annee : 2000 + annee if annee < 100 else annee)
df_2006["an"] = df_2006["an"].apply(lambda annee : 2000 + annee if annee < 100 else annee)
df_2007["an"] = df_2007["an"].apply(lambda annee : 2000 + annee if annee < 100 else annee)
df_2008["an"] = df_2008["an"].apply(lambda annee : 2000 + annee if annee < 100 else annee)
df_2009["an"] = df_2009["an"].apply(lambda annee : 2000 + annee if annee < 100 else annee)
df_2010["an"] = df_2010["an"].apply(lambda annee : 2000 + annee if annee < 100 else annee)
df_2011["an"] = df_2011["an"].apply(lambda annee : 2000 + annee if annee < 100 else annee)
df_2012["an"] = df_2012["an"].apply(lambda annee : 2000 + annee if annee < 100 else annee)
df_2013["an"] = df_2013["an"].apply(lambda annee : 2000 + annee if annee < 100 else annee)
df_2014["an"] = df_2014["an"].apply(lambda annee : 2000 + annee if annee < 100 else annee)
df_2015["an"] = df_2015["an"].apply(lambda annee : 2000 + annee if annee < 100 else annee)
df_2016["an"] = df_2016["an"].apply(lambda annee : 2000 + annee if annee < 100 else annee)
df_2017["an"] = df_2017["an"].apply(lambda annee : 2000 + annee if annee < 100 else annee)
df_2018["an"] = df_2018["an"].apply(lambda annee : 2000 + annee if annee < 100 else annee)
df_2019["an"] = df_2019["an"].apply(lambda annee : 2000 + annee if annee < 100 else annee)
df_2020["an"] = df_2020["an"].apply(lambda annee : 2000 + annee if annee < 100 else annee)
df_2021["an"] = df_2021["an"].apply(lambda annee : 2000 + annee if annee < 100 else annee)
df_2022["an"] = df_2022["an"].apply(lambda annee : 2000 + annee if annee < 100 else annee)


In [4]:
# 2. Variable 'hrmn', retirer le caractère ':' et forcer le type en int sur fichiers de 2019 à 2022
df_2019['hrmn'] = df_2019['hrmn'].astype(str).str.replace(':', '').astype(int)
df_2020['hrmn'] = df_2020['hrmn'].astype(str).str.replace(':', '').astype(int)
df_2021['hrmn'] = df_2021['hrmn'].astype(str).str.replace(':', '').astype(int)
df_2022['hrmn'] = df_2022['hrmn'].astype(str).str.replace(':', '').astype(int)

# Opération similaire mais à priori intule sur les autres fichiers
df_2005['hrmn'] = df_2005['hrmn'].astype(str).str.replace(':', '').astype(int)
df_2006['hrmn'] = df_2006['hrmn'].astype(str).str.replace(':', '').astype(int)
df_2007['hrmn'] = df_2007['hrmn'].astype(str).str.replace(':', '').astype(int)
df_2008['hrmn'] = df_2008['hrmn'].astype(str).str.replace(':', '').astype(int)
df_2009['hrmn'] = df_2009['hrmn'].astype(str).str.replace(':', '').astype(int)
df_2010['hrmn'] = df_2010['hrmn'].astype(str).str.replace(':', '').astype(int)
df_2011['hrmn'] = df_2011['hrmn'].astype(str).str.replace(':', '').astype(int)
df_2012['hrmn'] = df_2012['hrmn'].astype(str).str.replace(':', '').astype(int)
df_2013['hrmn'] = df_2013['hrmn'].astype(str).str.replace(':', '').astype(int)
df_2014['hrmn'] = df_2014['hrmn'].astype(str).str.replace(':', '').astype(int)
df_2015['hrmn'] = df_2015['hrmn'].astype(str).str.replace(':', '').astype(int)
df_2016['hrmn'] = df_2016['hrmn'].astype(str).str.replace(':', '').astype(int)
df_2017['hrmn'] = df_2017['hrmn'].astype(str).str.replace(':', '').astype(int)
df_2018['hrmn'] = df_2018['hrmn'].astype(str).str.replace(':', '').astype(int)


In [5]:
# 3. Variable 'gps' manquante à ajouter aux fichier de 2019 à 2022 en laissant Nan comme valeur
df_2019['gps'] = None
df_2020['gps'] = None
df_2021['gps'] = None
df_2022['gps'] = None


In [6]:
# 4. Renommer la colonne  'Num_Acc' en 'id_accident' (sauf sur le fichier de 2022
rename_columns = {'Num_Acc' : 'id_accident'}
df_2005 = df_2005.rename(columns = rename_columns)
df_2006 = df_2006.rename(columns = rename_columns)
df_2007 = df_2007.rename(columns = rename_columns)
df_2008 = df_2008.rename(columns = rename_columns)
df_2009 = df_2009.rename(columns = rename_columns)
df_2010 = df_2010.rename(columns = rename_columns)
df_2011 = df_2011.rename(columns = rename_columns)
df_2012 = df_2012.rename(columns = rename_columns)
df_2013 = df_2013.rename(columns = rename_columns)
df_2014 = df_2014.rename(columns = rename_columns)
df_2015 = df_2015.rename(columns = rename_columns)
df_2016 = df_2016.rename(columns = rename_columns)
df_2017 = df_2017.rename(columns = rename_columns)
df_2018 = df_2018.rename(columns = rename_columns)
df_2019 = df_2019.rename(columns = rename_columns)
df_2020 = df_2020.rename(columns = rename_columns)
df_2021 = df_2021.rename(columns = rename_columns)

rename_columns = {'Accident_Id' : 'id_accident'}
df_2022 = df_2022.rename(columns = rename_columns)


In [7]:
# 5. Coordonnées GPS --> transcoder en float64 et réajuster les valeurs de 2005 à 2018
# lattitude et longitude : 2005 à 2018 (hors 2009) --> diviser la valeur par 100000
df_2005['gps_lat'] = df_2005['lat'] / 100000
df_2005['gps_long'] = df_2005['long'] / 100000

df_2006['gps_lat'] = df_2006['lat'] / 100000
df_2006['gps_long'] = df_2006['long'] / 100000

df_2007['gps_lat'] = df_2007['lat'] / 100000
df_2007['gps_long'] = df_2007['long'] / 100000

df_2008['gps_lat'] = df_2008['lat'] / 100000
df_2008['gps_long'] = df_2008['long'] / 100000

df_2009['gps_lat'] = None
df_2009['gps_long'] = None

df_2010['gps_lat'] = df_2010['lat'] / 100000
df_2010['gps_long'] = df_2010['long'] / 100000

df_2011['gps_lat'] = df_2011['lat'] / 100000
df_2011['gps_long'] = df_2011['long'] / 100000

df_2012['gps_lat'] = df_2012['lat'] / 100000
df_2012['gps_long'] = df_2012['long'] / 100000

df_2013['gps_lat'] = df_2013['lat'] / 100000
df_2013['gps_long'] = df_2013['long'] / 100000

df_2014['gps_lat'] = df_2014['lat'] / 100000
df_2014['gps_long'] = df_2014['long'] / 100000

df_2015['gps_lat'] = df_2015['lat'] / 100000
df_2015['gps_long'] = df_2015['long'] / 100000

df_2016['gps_lat'] = df_2016['lat'] / 100000
df_2016['gps_long'] = df_2016['long'] / 100000

df_2017['gps_lat'] = df_2017['lat'] / 100000
df_2017['gps_long'] = df_2017['long'] / 100000

df_2018['gps_lat'] = df_2018['lat'] / 100000
df_2018['gps_long'] = df_2018['long'] / 100000

# Transcode les coordonnées gps de 2019 à 2022 en float64
df_2019['gps_lat'] = df_2019['lat'].astype('str').str.replace(',', '.').astype('float')
df_2019['gps_long'] = df_2019['long'].astype('str').str.replace(',', '.').astype('float')

df_2020['gps_lat'] = df_2020['lat'].astype('str').str.replace(',', '.').astype('float')
df_2020['gps_long'] = df_2020['long'].astype('str').str.replace(',', '.').astype('float')

df_2021['gps_lat'] = df_2021['lat'].astype('str').str.replace(',', '.').astype('float')
df_2021['gps_long'] = df_2021['long'].astype('str').str.replace(',', '.').astype('float')

df_2022['gps_lat'] = df_2022['lat'].astype('str').str.replace(',', '.').astype('float')
df_2022['gps_long'] = df_2022['long'].astype('str').str.replace(',', '.').astype('float')


In [8]:
# 6.1 - 2022 : une commune associée au code 'N/C' --> il s'agit en fait de la commune 500 (cf.coordonnées GPS)
df_2022.loc[df_2022['com'] == 'N/C', 'com'] = '14500'
df_2022.loc[df_2022['com'] == '#ERREUR', 'com'] = '14500'

# 6.2 Commune - de 2019 à 2022 : garder les 3 derniers caractères de droite et transcoder en 'float64'
df_2019['com'] = df_2019['com'].apply(lambda x: str(x)[2:] if len(str(x)) == 5 else x).astype('float')
df_2020['com'] = df_2020['com'].apply(lambda x: str(x)[2:] if len(str(x)) == 5 else x).astype('float')
df_2021['com'] = df_2021['com'].apply(lambda x: str(x)[2:] if len(str(x)) == 5 else x).astype('float')
df_2022['com'] = df_2022['com'].apply(lambda x: str(x)[2:] if len(str(x)) == 5 else x).astype('float')

# 6.3 Commune - de 2005 à 2018 : maintien en l'état
df_2005.head()
df_2022[(df_2022.id_accident == 202200000519)]




,id_accident,jour,mois,an,hrmn,lum,dep,com,agg,int,atm,col,adr,lat,long,gps,gps_lat,gps_long
518,202200000519,31,12,2022,1505,1,14,500.0,2,6,1,6,D577,"48,8643370000","-0,8733480000",None,48.864337,-0.873348


In [9]:
# 7. Département
#   - de 2005 à 2018 : diviser par 10 si %10 sinon laisser en l'état (Corse=201/202, DOM=XXX)
df_2005['dept'] = df_2005['dep'].apply(lambda valeur : valeur/10 if (valeur%10==0) else valeur).astype(int)
df_2006['dept'] = df_2006['dep'].apply(lambda valeur : valeur/10 if (valeur%10==0) else valeur).astype(int)
df_2007['dept'] = df_2007['dep'].apply(lambda valeur : valeur/10 if (valeur%10==0) else valeur).astype(int)
df_2008['dept'] = df_2008['dep'].apply(lambda valeur : valeur/10 if (valeur%10==0) else valeur).astype(int)
df_2009['dept'] = df_2009['dep'].apply(lambda valeur : valeur/10 if (valeur%10==0) else valeur).astype(int)
df_2010['dept'] = df_2010['dep'].apply(lambda valeur : valeur/10 if (valeur%10==0) else valeur).astype(int)
df_2011['dept'] = df_2011['dep'].apply(lambda valeur : valeur/10 if (valeur%10==0) else valeur).astype(int)
df_2012['dept'] = df_2012['dep'].apply(lambda valeur : valeur/10 if (valeur%10==0) else valeur).astype(int)
df_2013['dept'] = df_2013['dep'].apply(lambda valeur : valeur/10 if (valeur%10==0) else valeur).astype(int)
df_2014['dept'] = df_2014['dep'].apply(lambda valeur : valeur/10 if (valeur%10==0) else valeur).astype(int)
df_2015['dept'] = df_2015['dep'].apply(lambda valeur : valeur/10 if (valeur%10==0) else valeur).astype(int)
df_2016['dept'] = df_2016['dep'].apply(lambda valeur : valeur/10 if (valeur%10==0) else valeur).astype(int)
df_2017['dept'] = df_2017['dep'].apply(lambda valeur : valeur/10 if (valeur%10==0) else valeur).astype(int)
df_2018['dept'] = df_2018['dep'].apply(lambda valeur : valeur/10 if (valeur%10==0) else valeur).astype(int)
    
#   - de 2019 à 2022 : maintien en l'état en remplaçant les A en '01' et B en '02' (Corse=201/202 au lieu de 2A, 2B)
df_2019['dept'] = df_2019['dep'].apply(lambda valeur : str(valeur).replace('A', '01').replace('B', '02')).astype('int')
df_2020['dept'] = df_2020['dep'].apply(lambda valeur : str(valeur).replace('A', '01').replace('B', '02')).astype('int')
df_2021['dept'] = df_2021['dep'].apply(lambda valeur : str(valeur).replace('A', '01').replace('B', '02')).astype('int')
df_2022['dept'] = df_2022['dep'].apply(lambda valeur : str(valeur).replace('A', '01').replace('B', '02')).astype('int')


#df_2019.head()

In [10]:
# 8. Réindexer les colonnes de chaque df dans le même ordre
ordre_colonnes = ['id_accident', 'an', 'mois', 'jour', 'hrmn', 'lum', 'agg', 'int', 'atm', 'col', 
                  'com', 'adr', 'gps', 'lat', 'gps_lat', 'long', 'gps_long', 'dept', 'dep']
df_2005 = df_2005.reindex(columns=ordre_colonnes)
df_2006 = df_2006.reindex(columns=ordre_colonnes)
df_2007 = df_2007.reindex(columns=ordre_colonnes)
df_2008 = df_2008.reindex(columns=ordre_colonnes)
df_2009 = df_2009.reindex(columns=ordre_colonnes)
df_2010 = df_2010.reindex(columns=ordre_colonnes)
df_2011 = df_2011.reindex(columns=ordre_colonnes)
df_2012 = df_2012.reindex(columns=ordre_colonnes)
df_2013 = df_2013.reindex(columns=ordre_colonnes)
df_2014 = df_2014.reindex(columns=ordre_colonnes)
df_2015 = df_2015.reindex(columns=ordre_colonnes)
df_2016 = df_2016.reindex(columns=ordre_colonnes)
df_2017 = df_2017.reindex(columns=ordre_colonnes)
df_2018 = df_2018.reindex(columns=ordre_colonnes)
df_2019 = df_2019.reindex(columns=ordre_colonnes)
df_2020 = df_2020.reindex(columns=ordre_colonnes)
df_2021 = df_2021.reindex(columns=ordre_colonnes)
df_2022 = df_2022.reindex(columns=ordre_colonnes)


In [11]:
# 9. Concaténer les df en un seul
carac = df_2005.copy()
carac = pd.concat ([carac, df_2006], axis = 0, copy=True)
carac = pd.concat ([carac, df_2007], axis = 0, copy=True)
carac = pd.concat ([carac, df_2008], axis = 0, copy=True)
carac = pd.concat ([carac, df_2009], axis = 0, copy=True)
carac = pd.concat ([carac, df_2010], axis = 0, copy=True)
carac = pd.concat ([carac, df_2011], axis = 0, copy=True)
carac = pd.concat ([carac, df_2012], axis = 0, copy=True)
carac = pd.concat ([carac, df_2013], axis = 0, copy=True)
carac = pd.concat ([carac, df_2014], axis = 0, copy=True)
carac = pd.concat ([carac, df_2015], axis = 0, copy=True)
carac = pd.concat ([carac, df_2016], axis = 0, copy=True)
carac = pd.concat ([carac, df_2017], axis = 0, copy=True)
carac = pd.concat ([carac, df_2018], axis = 0, copy=True)
carac = pd.concat ([carac, df_2019], axis = 0, copy=True)
carac = pd.concat ([carac, df_2020], axis = 0, copy=True)
carac = pd.concat ([carac, df_2021], axis = 0, copy=True)
carac = pd.concat ([carac, df_2022], axis = 0, copy=True)


/var/folders/66/kgr5_fnx18s10y16833g5x600000gn/T/ipykernel_85513/509947701.py:6: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  carac = pd.concat ([carac, df_2009], axis = 0, copy=True)


In [12]:
# 10. Controles et ajustements

# 10.1 'atm' : 73 NaN --> -1=Non renseigné et transcodage de float64 --> int64
carac['atm'] = carac['atm'].fillna(-1).astype('int')

# 10.2 'col' : 19 NaN --> -1=Non renseigné et transcodage de float64 --> int64
carac['col'] = carac['col'].fillna(-1).astype('int')

# 10.3 'com' : 2 Nan
carac[(carac.com.isna()==True)]

# 10.4 'int' : intersection renseignées à 0 alors que la valeur n'est pas référencée --> force ces 107 enreg à -1: Non renseigné
carac['int'] = carac['int'].apply(lambda valeur : -1 if valeur == 0 else valeur).astype('int')

# 10.5 'gps' : type la colonne en object pour éviter l'utilisation d'un dtype={'gps': 'string'} lors du pd.read_csv()
carac['gps'] = carac['gps'].astype('object')
                                   
carac['gps'].value_counts().sort_index()
# carac.head(50)
# carac.info()
#carac.lum.describe()

gps
0         9
A      7850
C         2
G      3387
M    462639
P         1
R      5316
S         4
T         2
Y       842
Name: count, dtype: int64

In [52]:
# 11. Suppression des colonnes inutiles
if 'lat' in carac.columns :
    drop_columns = ['lat', 'long', 'dep']
    carac = carac.drop(drop_columns, axis = 1)

    #df_2022 = df_2022.drop(drop_columns, axis = 1)

else :
    display('>>> Colonnes déjà supprimées du DataFrame.')


In [53]:
# 12. Sauvegarde du fichier en csv (sans index) 
carac.to_csv(repDataConcat + "caracteristiques.csv", index=False, encoding='utf-8')

# 12.2 Sauvegarde du fichier en csv 2022 (sans index) 
#df_2022.to_csv(repDataConcat + "caracteristiques_2022.csv", index=False, encoding='utf-8')